# Story Scape
- StoryScape is an AI-powered app for kids that creates captivating story videos based on user prompts. Unlock your imagination and dive into a world of endless storytelling with StoryScape.

Install the required libraries

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    ! pip3 install --upgrade pip
    ! pip3 install torchvision==0.14.1
    ! pip3 install transformers==4.27.1
    ! pip3 install diffusers==0.15.1
    ! pip3 install datasets==2.9.0
    ! pip3 install accelerate==0.18.0
    ! pip3 install triton==2.0.0.dev20221120
    ! pip3 install xformers==0.0.16
    # Install gdown for downloading example training images.
    ! pip3 install gdown
    # Remove wrong cublas version.
    ! pip3 uninstall nvidia_cublas_cu11 --yes

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
!pip install google-cloud-texttospeech
from google.cloud import storage
from google.cloud import texttospeech
import re
import vertexai
from vertexai.language_models import TextGenerationModel
!pip install moviepy
!pip install streamlit

Fill following variables for experiments environment:

In [ ]:
# Cloud project id.


PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

Initialize Vertex-AI API:

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Define constants

In [ ]:
# The pre-built training docker image. It contains training scripts and models.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/pytorch-diffusers-train:latest"

# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/pytorch-diffusers-serve"

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '<path to credentials file of your service account>'
!ls -l $GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
%%writefile app.py
import streamlit as st
import os
from google.cloud import storage
from google.cloud import texttospeech
import vertexai
from vertexai.language_models import TextGenerationModel
import re
from google.cloud import aiplatform
import torch
from diffusers import StableDiffusionPipeline
from moviepy.editor import VideoClip, AudioFileClip, ImageClip, VideoFileClip
from moviepy.video.compositing.concatenate import concatenate_videoclips
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
PROJECT_ID = ""  # @param {type:"string"}
# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}
# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)
PROJECT_ID = ""
vertexai.init(project=PROJECT_ID, location="")
# The pre-built training docker image. It contains training scripts and models.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/pytorch-diffusers-train:latest"

# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/pytorch-diffusers-serve"

import base64
import glob
import os
from datetime import datetime
from io import BytesIO

import requests
from google.cloud import aiplatform, storage
from PIL import Image


def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def base64_to_image(image_str):
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


def deploy_model(model_id, task):
    model_name = "stable-diffusion-v1"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_V100",
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint


def get_bucket_and_blob_name(filepath):
    # The gcs path is of the form gs://<bucket-name>/<blob-name>
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def upload_local_dir_to_gcs(local_dir_path, gcs_dir_path):
    """Uploads files in a local directory to a GCS directory."""
    client = storage.Client()
    bucket_name = gcs_dir_path.split("/")[2]
    bucket = client.get_bucket(bucket_name)
    for local_file in glob.glob(local_dir_path + "/**"):
        if not os.path.isfile(local_file):
            continue
        filename = local_file[1 + len(local_dir_path) :]
        gcs_file_path = os.path.join(gcs_dir_path, filename)
        _, blob_name = get_bucket_and_blob_name(gcs_file_path)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(local_file)
        print("Copied {} to {}.".format(local_file, gcs_file_path))



def generate_output(input_text):
    # Add your logic here to process the input text and generate the desired output
    input = f"You entered: {input_text}"
    return input

def story():
    st.title("Text Input App")
    input_text = st.text_input("Enter your text:")
    prompt = generate_output(input_text)
    if prompt.strip() != "":
        final_prompt=f"""Act as a story writer who specialize in children's stories. Your task is to write a 500 words story based on {prompt} and take help of additional tasks given below:
                    This story will have a proper title and a meaningful short moral
                    The story should be based on characters or things mentioned in {prompt} you should not assume other things.
                    If {prompt} doesn't have enough context create title using your intelligence
                    If in {prompt} the characters don't have name give them one fictional name
                    Your objective is to communicate a useful message for children to learn from  """
        vertexai.init(project="first-grove-392012", location="us-central1")
        parameters = {
            "temperature": 0.5,
            "max_output_tokens": 500,
            "top_p": 0.8,
            "top_k": 40
        }
        model = TextGenerationModel.from_pretrained("text-bison")
        response = model.predict(
          final_prompt,
            **parameters
        )
        return response.text
    else:
        return f"Prompt is empty. No story will be generated."

def divided_story(story_text):
    story = story_text.replace('\n', ' ')
    story = story_text.replace('*', ' ')
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', story)

    # Ensure there are at least two sentences in the story
    if len(sentences) < 2:
        raise ValueError("The story should contain at least two sentences.")

    # Group the sentences into pairs
    sentence_pairs = [sentences[i] + ' ' + sentences[i+1] for i in range(0, len(sentences) - 1, 2)]

    # If there is an odd number of sentences, keep the last sentence as it is
    if len(sentences) % 2 != 0:
        sentence_pairs.append(sentences[-1])

    return sentence_pairs

story_text = story()


def audio():
    story_ls = divided_story(story_text)
    project_id = ''
    client = storage.Client(project=project_id)
    bucket_name = 'bucket-innovaite'
    bucket = client.get_bucket(bucket_name)
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '<path to credentials file of your service account>'
    speech_client = texttospeech.TextToSpeechClient()
    # Delete existing files in the bucket
    blobs = bucket.list_blobs()
    for blob in blobs:
        blob.delete()
        print(f'Deleted file: {blob.name}')
    for i in range(len(story_ls)):
        input_text = texttospeech.SynthesisInput(text=story_ls[i])
        voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Studio-O",
        )

        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.LINEAR16,
            speaking_rate=1
        )

        response = speech_client.synthesize_speech(
            request={"input": input_text, "voice": voice, "audio_config": audio_config}
        )

        # The response's audio_content is binary.
        audio_filename=f"output{i+10}.mp3"
        with open(audio_filename, "wb") as out:
            out.write(response.audio_content)
            object_name_in_gcs_bucket = bucket.blob(audio_filename)
            object_name_in_gcs_bucket.upload_from_filename(audio_filename)


            print(f'Audio content written to file {audio_filename}')
def image():
    aud=audio()
    story_ls = divided_story(story_text)
    project_id = ''
    client = storage.Client(project=project_id)
    bucket_name = 'image_storygen'
    bucket = client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()
    for blob in blobs:
        blob.delete()
        print(f'Deleted file: {blob.name}')
    for i in range(len(story_ls)):
        prompt = f'''Generate an image that represents the scene: {story_ls[i]} with the help of the following task and context given below:
        1.Take the context by using summary of {story_ls} to generate best image for {story_ls[i]}
        2.Keep the image theme for {story_ls} vibrant, comic like,3D Cartoon Style.
        3.Do not include any text inside the image.
        4.Keep the same theme throughout the {story_ls}'''

        results = pipe(prompt=prompt, guidance_scale=25)
        images = results.images
        nsfw_detects = results.nsfw_content_detected

        # Save the image file
        image_filename = f"image{i+10}.png"
        images[0].save(image_filename)

        # Upload the image file to Google Cloud Storage
        object_name_in_gcs_bucket = bucket.blob(image_filename)
        object_name_in_gcs_bucket.upload_from_filename(image_filename)

        print(f'Image file {image_filename} saved and uploaded.')

def merge_audio_image(audio_bucket_name, image_bucket_name, output_file, video_bucket_name):
    img=image()
    storage_client = storage.Client()

    audio_bucket = storage_client.get_bucket(audio_bucket_name)
    image_bucket = storage_client.get_bucket(image_bucket_name)
    video_bucket = storage_client.get_bucket(video_bucket_name)

    audio_files = audio_bucket.list_blobs()
    image_files = image_bucket.list_blobs()

    audio_files_sorted = sorted(audio_files, key=lambda file: file.name)
    image_files_sorted = sorted(image_files, key=lambda file: file.name)

    video_clips = []
    for audio_file, image_file in zip(audio_files_sorted, image_files_sorted):
        audio_path = audio_file.name
        image_path = image_file.name

        audio_clip = AudioFileClip(audio_path)
        image_clip = ImageClip(image_path)

        # Set the duration of the image clip to match the audio clip's duration
        image_clip = image_clip.set_duration(audio_clip.duration)

        # Set the audio of the image clip to match the audio clip
        image_clip = image_clip.set_audio(audio_clip)

        # Append the image clip to the video clips list
        video_clips.append(image_clip)

    # Concatenate the video clips to create the final video
    final_video = concatenate_videoclips(video_clips)
    # Write the final video to the output file
    final_video.write_videofile(output_file, codec="libx264", audio_codec="aac", fps=24)

    # Upload the video file to the video bucket
    video_blob = video_bucket.blob(output_file)
    video_blob.upload_from_filename(output_file)
    video_bucket_name = "video_storygen"
    video_file_name = "output.mp4"



def main():
    # Specify the names of the audio, image, and video buckets, and the output file path
    audio_bucket_name = "bucket-innovaite"
    image_bucket_name = "image_storygen"
    video_bucket_name = "video_storygen"
    output_file = "output.mp4"
    vid= merge_audio_image(audio_bucket_name,image_bucket_name,output_file,video_bucket_name)
    if st.button("Generate Video"):
         st.spinner(text="Video in Progress")
         st.video("https://storage.cloud.google.com/video_storygen/output.mp4")
if __name__ == "__main__":
    main()




In [ ]:
!npm install localtunnel

In [ ]:
#paste the endpoint address before clicking on submit
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com